In [197]:
import pandas as pd
import re
import emoji
import string
from emoticon_lib import replace_emoticons_with_text
from deep_translator import (GoogleTranslator)
import html
import nltk
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import contractions
import numpy as np
from symspellpy import SymSpell, Verbosity

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [214]:
df = pd.read_csv('cyberbullying_tweets.csv')

DATA UNDERSTANDING

In [90]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [91]:
df.tail()

,tweet_text,cyberbullying_type
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity
47691,Bro. U gotta chill RT @CHILLShrammy: Dog FUCK ...,ethnicity


In [108]:
df.shape

(47692, 2)

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  object
dtypes: object(2)
memory usage: 745.3+ KB


In [94]:
df.describe()

,tweet_text,cyberbullying_type
count,47692,47692
unique,46017,6
top,RT @sailorhg: the intro for my hardware hackin...,religion
freq,2,7998


In [95]:
df.isna().sum()

tweet_text            0
cyberbullying_type    0
dtype: int64

In [96]:
# Unique values of target variable
df.cyberbullying_type.unique()

array(['not_cyberbullying', 'gender', 'religion', 'other_cyberbullying',
       'age', 'ethnicity'], dtype=object)

In [97]:
# Count of values of target variable
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

In [98]:
print('Num of Duplicated Rows:', df.duplicated().sum())

Num of Duplicated Rows: 36


DATA CLEANING

In [215]:
# Remove Duplicates
df =df.drop_duplicates(subset='tweet_text', keep="last")
df.shape

(46017, 2)

In [216]:
# Expand contraction
df['tweet_text'] = df['tweet_text'].apply(lambda e: contractions.fix(e))
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                                          | cyberbullying_type   |
|---:|:------------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words #katandandre, your food was crapilicious! #mkr                                                                       | not_cyberbullying    |
|  1 | Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc                 | not_cyberbullying    |
|  2 | @XochitlSuckkks a classy whore? Or more red velvet cupcakes?                                                                        | not_cyberbullying    |
|  4 | @RudhoeEnglish This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.                             | not_cyberbullying    |
|  5 | @Raja5aab

In [217]:
#Remove html tags
df['tweet_text'] = df['tweet_text'].apply(lambda e: html.unescape(e))

In [218]:
# Remove emoticons
df['tweet_text'] = df['tweet_text'].apply(lambda e: replace_emoticons_with_text(e))

In [219]:
# Remove email 
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"\S+@\S+(?:\.\S+)+", "", e))
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                                          | cyberbullying_type   |
|---:|:------------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words #katandandre, your food was crapilicious! #mkr                                                                       | not_cyberbullying    |
|  1 | Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc                 | not_cyberbullying    |
|  2 | @XochitlSuckkks a classy whore? Or more red velvet cupcakes?                                                                        | not_cyberbullying    |
|  4 | @RudhoeEnglish This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.                             | not_cyberbullying    |
|  5 | @Raja5aab

In [220]:
## Remove RT, #, URL and @: 
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r"(RT)|(?:\@|\#|https?|http?|www)\S+", "", e))
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                  | cyberbullying_type   |
|---:|:------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words  your food was crapilicious!                                                                 | not_cyberbullying    |
|  1 | Why is  so white?                                                                                           | not_cyberbullying    |
|  2 | a classy whore? Or more red velvet cupcakes?                                                                | not_cyberbullying    |
|  4 | This is an ISIS account pretending to be a Kurdish account.  Like Islam, it is all lies.                    | not_cyberbullying    |
|  5 | Yes, the test of god is that good or bad or indifferent or weird or whatever, it all proves gods existence. | not_cyberbullying    |
|  6 | Itu sekolah y

In [221]:
# Replace underscore
df['tweet_text'] = df['tweet_text'].apply(lambda e: e.replace('_', ' '))

In [222]:
# Change emojis to text
df['tweet_text'] = df['tweet_text'].apply(lambda e: emoji.demojize(e, language="en"))

In [224]:
# Replace underscore (space) and : (remove)
df['tweet_text'] = df['tweet_text'].apply(lambda e: e.replace('_', ' '))
df['tweet_text'] = df['tweet_text'].apply(lambda e: e.replace(':', ''))

In [226]:
# Word Segmentation  
import wordninja
df['tweet_text'] = df['tweet_text'].apply(lambda x: ' '.join(wordninja.split(x)))
df.head(10)    

,tweet_text,cyberbullying_type
0,In other words your food was crap i licious,not_cyberbullying
1,Why is so white,not_cyberbullying
2,a classy whore Or more red velvet cupcakes,not_cyberbullying
4,This is an IS IS account pretending to be a Ku...,not_cyberbullying
5,Yes the test of god is that good or bad or in ...,not_cyberbullying
6,It u sekolah ya buk an temp at bully Ga j a uh...,not_cyberbullying
7,Karma I hope it bites Kat on the butt She is j...,not_cyberbullying
9,Rebecca Black Drops Out of School Due to Bullying,not_cyberbullying
10,annoyed undecided uneasy or hesitant t co Us Q...,not_cyberbullying
11,The Bully flushes on K D annoyed undecided une...,not_cyberbullying


In [24]:
# Remove digits
#df['tweet_text'] = df['tweet_text'].replace(to_replace=r'\d', value='', regex=True)

In [228]:
# Drop rows with nan or empty spaces 
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
df = df[df['tweet_text'].notna()]
df.shape

(45910, 2)

In [229]:
# Remove special characters and digits
df['tweet_text'] = df['tweet_text'].apply(lambda e: re.sub(r'[^a-zA-Z ]', '', e))

In [ ]:
# Translate
#df['tweet_text'] = df['tweet_text'].apply(lambda e: GoogleTranslator(source='auto', target='en').translate(text = e))

In [ ]:
# Remove punctuations
# def remove_punctuations(text):
#     remove_punc = [w for w in text if w not in string.punctuation]
#     remove_punc = ''.join(remove_punc)
#     return ''.join([wrd for wrd in remove_punc.split()])
# df['tweet_text'] = df['tweet_text'].apply(remove_punctuations)

In [230]:
# Remove extra whitespaces after removing noise and punctuations
def remove_extra_whitespaces(text):
    # Replace multiple consecutive whitespaces with a single space
    text = re.sub(r'\s+', ' ', text) 
    return text.strip()  # Remove leading and trailing whitespaces
df['tweet_text'] = df['tweet_text'].apply(remove_extra_whitespaces)

In [232]:
# Normalize Words / Remove repeating words with 1 or  (use regex) 
def remove_words_with_one_char_or_repeating_chars(text):   
    text = re.sub(r'\b\w\b', '', text)   # Remove words with 1 character
    text = re.sub(r'\b\w*(\w)\1{2,}\w*\b', '', text)   # Remove words with 3 or more repeating characters
    return text
df['tweet_text'] = df['tweet_text'].apply(remove_words_with_one_char_or_repeating_chars)
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                | cyberbullying_type   |
|---:|:----------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | In other words your food was crap  licious                                                                | not_cyberbullying    |
|  1 | Why is so white                                                                                           | not_cyberbullying    |
|  2 | classy whore Or more red velvet cupcakes                                                                  | not_cyberbullying    |
|  4 | This is an IS IS account pretending to be  Kurdish account Like Islam it is all lies                      | not_cyberbullying    |
|  5 | Yes the test of god is that good or bad or in different or weird or whatever it all proves gods existence | not_cyberbullying    |
|  6 | It  sekolah ya buk an temp 

In [233]:
# Convert to lower case
df['tweet_text'] = df['tweet_text'].str.lower()
df

,tweet_text,cyberbullying_type
0,in other words your food was crap licious,not_cyberbullying
1,why is so white,not_cyberbullying
2,classy whore or more red velvet cupcakes,not_cyberbullying
4,this is an is is account pretending to be kur...,not_cyberbullying
5,yes the test of god is that good or bad or in ...,not_cyberbullying
...,...,...
47687,black people are not expected to do anything d...,ethnicity
47688,turner did not withhold his disappointment tur...,ethnicity
47689,swear to god this dumb nigger bitch have got...,ethnicity
47690,yea fuck you if you are nigger fucking un fol...,ethnicity


In [234]:
# Tokenization
def tokenize_text(text):
    if isinstance(text, str):  # Check if text is a string
        tokens = word_tokenize(text)
        return tokens
    else:
        return []  # Return an empty list for NaN values

df['tweet_text'] = df['tweet_text'].apply(tokenize_text)
print(df.head(10).to_markdown())

|    | tweet_text                                                                                                                                                                   | cyberbullying_type   |
|---:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | ['in', 'other', 'words', 'your', 'food', 'was', 'crap', 'licious']                                                                                                           | not_cyberbullying    |
|  1 | ['why', 'is', 'so', 'white']                                                                                                                                                 | not_cyberbullying    |
|  2 | ['classy', 'whore', 'or', 'more', 'red', 'velvet', 'cupcakes']                                                                                                               

In [235]:
# Remove stopwords
stopword = stopwords.words('english')
def remove_stopwords(tokens): 
    #tokens = word_tokenize(text)
    non_stopwords = [word for word in tokens if word not in stopword]
    return non_stopwords

df['tweet_text'] = df['tweet_text'].apply(remove_stopwords)
print(df.head(10).to_markdown())

|    | tweet_text                                                                                             | cyberbullying_type   |
|---:|:-------------------------------------------------------------------------------------------------------|:---------------------|
|  0 | ['words', 'food', 'crap', 'licious']                                                                   | not_cyberbullying    |
|  1 | ['white']                                                                                              | not_cyberbullying    |
|  2 | ['classy', 'whore', 'red', 'velvet', 'cupcakes']                                                       | not_cyberbullying    |
|  4 | ['account', 'pretending', 'kurdish', 'account', 'like', 'islam', 'lies']                               | not_cyberbullying    |
|  5 | ['yes', 'test', 'god', 'good', 'bad', 'different', 'weird', 'whatever', 'proves', 'gods', 'existence'] | not_cyberbullying    |
|  6 | ['sekolah', 'ya', 'buk', 'temp', 'bully', 'ga', 

In [236]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
def lemmatize_words(tokens):
    return ' '.join([lemmatizer.lemmatize(word) for word in tokens])
df["tweet_text"] = df["tweet_text"].apply(lemmatize_words)
print(df.head(10).to_markdown())

|    | tweet_text                                                        | cyberbullying_type   |
|---:|:------------------------------------------------------------------|:---------------------|
|  0 | word food crap licious                                            | not_cyberbullying    |
|  1 | white                                                             | not_cyberbullying    |
|  2 | classy whore red velvet cupcake                                   | not_cyberbullying    |
|  4 | account pretending kurdish account like islam lie                 | not_cyberbullying    |
|  5 | yes test god good bad different weird whatever prof god existence | not_cyberbullying    |
|  6 | sekolah ya buk temp bully ga uh kaya ne raka                      | not_cyberbullying    |
|  7 | karma hope bite kat butt nasty                                    | not_cyberbullying    |
|  9 | rebecca black drop school due bullying                            | not_cyberbullying    |
| 10 | annoyed undec

In [238]:
# Remove any non english words
english_words = set(words.words())

def remove_non_english(tokens):
    english_tokens = []
    for word in tokens:
        if word in english_words:
            english_tokens.append(word)
        else:
            english_tokens.append('')
    return [token for token in english_tokens if token != '']

df['tweet_text'] = df['tweet_text'].apply(tokenize_text)
df['tweet_text'] = df['tweet_text'].apply(remove_non_english)

In [239]:
# Data shape after removing non english words
print(df.head(10).to_markdown())
df.shape

|    | tweet_text                                                                                          | cyberbullying_type   |
|---:|:----------------------------------------------------------------------------------------------------|:---------------------|
|  0 | ['word', 'food', 'crap']                                                                            | not_cyberbullying    |
|  1 | ['white']                                                                                           | not_cyberbullying    |
|  2 | ['classy', 'whore', 'red', 'velvet', 'cupcake']                                                     | not_cyberbullying    |
|  4 | ['account', 'account', 'like', 'lie']                                                               | not_cyberbullying    |
|  5 | ['yes', 'test', 'god', 'good', 'bad', 'different', 'weird', 'whatever', 'prof', 'god', 'existence'] | not_cyberbullying    |
|  6 | ['ya', 'temp', 'bully', 'ga', 'ne']                                  

(45910, 2)

In [242]:
# Join the tokenized tweet_text to check for duplicated rows
df['tweet_text'] = df['tweet_text'].apply(lambda tokens: ' '.join(tokens))
print("Total number of duplicated rows: ", df.duplicated().sum())

,tweet_text,cyberbullying_type
0,word food crap,not_cyberbullying
1,white,not_cyberbullying
2,classy whore red velvet cupcake,not_cyberbullying
4,account account like lie,not_cyberbullying
5,yes test god good bad different weird whatever...,not_cyberbullying
6,ya temp bully ga ne,not_cyberbullying
7,karma hope bite kat butt nasty,not_cyberbullying
9,black drop school due bullying,not_cyberbullying
10,undecided uneasy hesitant u,not_cyberbullying
11,bully flush undecided uneasy hesitant twit,not_cyberbullying


In [261]:
# Remove duplicate rows
duplicate_rows = df[df.duplicated(keep=False)]
duplicate_rows

,tweet_text,cyberbullying_type
26,bullying,not_cyberbullying
31,,not_cyberbullying
39,,not_cyberbullying
55,,not_cyberbullying
83,bullying contra o sol,not_cyberbullying
...,...,...
47661,fat white bitch dumb nigger,ethnicity
47663,get dick tonight mad black man dumb nigger,ethnicity
47670,scale cracker pretzel salty dumb nigger,ethnicity
47674,ya dumb nigger,ethnicity


In [267]:
# Remove duplicate rows
df.drop_duplicates(subset=["tweet_text"], inplace=True)
df.shape

(43766, 2)

In [272]:
# After removing duplicated rows
df.duplicated().sum()

0

In [266]:
df[df['tweet_text'] == '']

,tweet_text,cyberbullying_type
40395,yea nigger un follow dumb nigger,ethnicity
47690,yea nigger un follow dumb nigger,ethnicity


In [270]:
df[df['tweet_text'] == '']

,tweet_text,cyberbullying_type
31,,not_cyberbullying


In [271]:
df = df[df['tweet_text'] != '']
df.shape

(43765, 2)

In [273]:
# Save joined dataset into new csv file
df.to_csv('joined_cyberbullying_tweet.csv', index=False)

In [274]:
df_tokenized = df.copy()
df_tokenized['tweet_text'] = df_tokenized['tweet_text'].apply(tokenize_text)
df_tokenized

,tweet_text,cyberbullying_type
0,"[word, food, crap]",not_cyberbullying
1,[white],not_cyberbullying
2,"[classy, whore, red, velvet, cupcake]",not_cyberbullying
4,"[account, account, like, lie]",not_cyberbullying
5,"[yes, test, god, good, bad, different, weird, ...",not_cyberbullying
...,...,...
47686,"[shut, dumb, acting, like, nagging, nigger]",ethnicity
47687,"[black, people, anything, anything, yet, free,...",ethnicity
47688,"[turner, withhold, disappointment, turner, cou...",ethnicity
47689,"[swear, god, dumb, nigger, bitch, got, bleach,...",ethnicity


In [275]:
# Save tokenized preprocessed dataset into new csv file
df_tokenized.to_csv('tokenized_cyberbullying_tweet.csv', index=False)

In [280]:
df1 = pd.read_csv('joined_cyberbullying_tweet.csv')
df1.duplicated().sum()

0